In [ ]:
# ======================================================
# Project: Food Collector
# Authors: Gamaliel Marines Olvera A01708746
#          Juan Pablo Cabrera Quiroga
#          Sebastian Flores 
# Description: This script contains the implementation
#              of the Food Collector simulation and 
#              the server that allows the communication
#              between the simulation and the Unity
#              application.
# ======================================================

In [ ]:
# ======================================================
# Imports
# ======================================================


from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.colors import ListedColormap

import numpy as np
import random

from http.server import BaseHTTPRequestHandler, HTTPServer
import logging
import json

import pandas as pd
import math

In [ ]:
#=============================================
# Food Agent
#=============================================

class food(Agent):
    def __init__(self, id, model, foodAmount):
        super().__init__(id, model)
        self.type = 1
        self.foodAmount = foodAmount
        self.active = True


    

In [ ]:
#=============================================
# Search Agent
#=============================================
class searchAgent(Agent):
    def __init__(self, id, model):
        super().__init__(id, model)
        self.type = 2
        self.active = True

    # Function to move randomly
    def moveRandom(self):
        possible_steps = self.model.grid.get_neighborhood(self.pos, moore=False, include_center=False)
        possible_steps_list = list(possible_steps)

        for step in possible_steps_list:
            if self.model.foods[step[0]][step[1]] != 0:
                possible_steps_list.remove(step)

        new_position = self.random.choice(possible_steps_list)
        self.model.grid.move_agent(self, new_position)
        





In [ ]:
#=============================================
# Carrier Agent
#=============================================
class carrierAgent(Agent):
    def __init__(self, id, model):
        super().__init__(id, model)
        self.type = 3
        self.food_carried = False
        self.active = True

    def move2food():
        return
    
    def move2warehouse():
        return
        
    def pickfood():
        return

    def dropfood():
        return

    def step(self):
        if not self.food_carried:
            self.move2food()
        elif self.food_carried:
            self.move2warehouse()



In [ ]:
# ======================================================
# Function: get_grid
# Description: Gets the grid of the model.
# Parameters:  model: The model where the grid belongs.
# Return: The grid of the model.
# ======================================================
def get_grid(model):
  grid = [[0 for x in range(model.grid.height)] for y in range(model.grid.width)]
  for (content, (x, y)) in model.grid.coord_iter():
    if content:
        cell_type = 0
        for agent in content:
            cell_type = agent.type
        grid[x][y] = cell_type

  return grid

In [ ]:
# ======================================================
# Food Collector Model Class
# ======================================================
class foodCollector(Model):
    # ==============================================================
    # Function: __init__
    # Description: Initializes the food collector model.
    # Parameters:  width: The width of the grid.
    #              height: The height of the grid.
    #              map: The map of the simulation.
    #              num_robots: The number of robots in the simulation.
    # Return: None
    # ==============================================================
    def __init__(self, width, height, map, num_robots):
        self.grid = MultiGrid(width, height, torus = False)
        self.schedule = RandomActivation(self)
        self.current_id = 0
        self.robot_initial_pos = (0, 0)
        self.datacollector = DataCollector(model_reporters={"Grid" : get_grid})
        self.robotId = []
        self.visited_positions = []
        self.foods = set()
        self.obstacles = set()
        self.all_positions = [(x, y) for x in range(width) for y in range(height)]
        self.steps = 0
        self.scouting = False
        self.clean = False
        self.food_groups = []
        self.num_robots = num_robots
        self.robots_finished = set()
        self.left_foodes = set()

In [ ]:
# ======================================================
# Server Class
# ======================================================
class Server(BaseHTTPRequestHandler):
    # ==============================================================
    # Function: _set_response
    # Description: Sets the response of the server.
    # Parameters:  None
    # Return: None
    # ==============================================================
    def _set_response(self):
        self.send_response(200)
        self.send_header('Content-type', 'application/json')
        self.end_headers()
        
    # ==============================================================
    # Function: do_GET
    # Description: Handles the GET request.
    # Parameters:  None
    # Return: None
    # ==============================================================
    def do_GET(self):
        self._set_response()
        self.wfile.write("GET request for {}".format(self.path).encode('utf-8'))

    # ==============================================================
    # Function: do_POST
    # Description: Handles the POST request and controls the 
    #              simulation´s steps.
    # Parameters:  None
    # Return: None
    # ==============================================================
    def do_POST(self):
        if not model.clean:
            model.step()
            grid = json.dumps(get_grid(model))
            print(grid)
            self._set_response()
            self.wfile.write(str(grid).encode('utf-8'))
        else:
            self._set_response()
            self.wfile.write("Simulation finished!".encode('utf-8'))

# ======================================================
# Function: run
# Description: Runs the server.
# Parameters:  server_class: The server class.
#              handler_class: The handler class.
#              port: The port where the server will run.
# Return: None
# ======================================================
def run(server_class=HTTPServer, handler_class=Server, port=8585):
    logging.basicConfig(level=logging.INFO)
    server_address = ('', port)
    httpd = server_class(server_address, handler_class)
    logging.info("Starting httpd...\n") # HTTPD is HTTP Daemon!
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:   # CTRL+C stops the server
        pass
    httpd.server_close()
    logging.info("Stopping httpd...\n")

# ======================================================
# Main
# ======================================================
map = [item.split() for item in map.split('\n')]
data = map.pop(0)
width, height = int(data[0]), int(data[1])

num_robots = 5

model = foodCollector(width, height, map, num_robots)

run()
